# The Azure AI Foundry SDK
The Azure AI Foundry SDK is a comprehensive toolchain designed to simplify the development of AI applications on Azure. It enables developers to:

- Access popular models from various model providers through a single interface
- Easily combine together models, data, and AI services to build AI-powered applications
- Evaluate, debug, and improve application quality & safety across development, testing, and production environments

The Azure AI Foundry SDK is a set of packages and services designed to work together. You can use the Azure AI Projects client library to easily use multiple services through a single project client and connection string. You can also use services and SDKs on their own and connect directly to your services.

# Setup Azure AI Foundry  
To start using Foundry, you need to create a dedicated resource and launch a new project. Follow the steps below.

### 1 Creating the Azure AI Foundry Resource  
- In the Azure portal create a new **Azure AI Foundry Hub** resource within your Resource Group.  
- Access the resource and click on **“Launch Azure AI Foundry”** to open the dedicated interface.

### 2 Creating a New Foundry Project  
- Within the Azure AI Foundry interface, create a **new project**.

### 3 Deploying the Model  
- Inside the project, deploy the gpt-4o model.  
- Make sure to set the following parameters:
  - **Model name**: gpt-4o  
  - **Deployment Type**: Data Zone Standard  
  - **Model Version**: 2024-08-06

### 4 Configuring Tracing  
- Within the Foundry project, go to the **Tracing** section.  
- Click **Create New** to create a new **Application Insights** resource.

### 5 Connecting the Foundry Project to the Notebook  
- In the `credentials.env` file, add the **connection string** of the new project (`AI_PROJECT_CONNECTION_STRING`), the **endpoint** (`AZURE_OPENAI_ENDPOINT`), and the **key** (`AZURE_OPENAI_API_KEY`) of the deployed OpenAI model.


# Connect to your AI Project

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path='../infra/credentials.env')

# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
project_connection_string=os.environ["AI_PROJECT_CONNECTION_STRING"]

# Construct AI project client
azure_ai_project_string = AIProjectClient.from_connection_string(
  conn_str=project_connection_string,
  credential=DefaultAzureCredential())


project = AIProjectClient.from_connection_string(
  conn_str=project_connection_string,
  credential=DefaultAzureCredential())

# Use model inference API to call an Azure OpenAI model client

In [2]:
openai = project.inference.get_azure_openai_client(api_version=os.environ["AZURE_OPENAI_API_VERSION"])
response = openai.chat.completions.create(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    messages=[
        {"role": "system", "content": "You are a helpful writing assistant that works for an Italian university faculty specialized in AI"},
        {"role": "user", "content": "Write me 3 short sentences about the AI trends in 2025"},
    ]
)

print(response.choices[0].message.content)

1. In 2025, AI is expected to heavily integrate with augmented reality applications, enhancing real-time decision-making in industries such as healthcare and manufacturing.
   
2. The development of AI systems capable of unsupervised learning will lead to breakthroughs in adaptive technologies, significantly pushing the boundaries of personalization and user interaction.

3. Ethical AI will become a major focal point, with increased scrutiny on data privacy and transparency driving innovations in AI governance and regulatory frameworks.


# Use model inference API to call a generic chat completion client (in this case, also Azure OpenAI)

In [3]:
# get an chat inferencing client using the project's default model inferencing endpoint
chat = project.inference.get_chat_completions_client()

# run a chat completion using the inferencing client
response = chat.complete(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    messages=[
        {"role": "system", "content": "You are a helpful writing assistant that works for an Italian university faculty specialized in AI"},
        {"role": "user", "content": "Write me 3 short sentences about the AI trends in 2025"},
    ]
)

print(response.choices[0].message.content)

1. In 2025, AI is expected to enhance human-machine collaboration through natural language processing advancements, allowing for seamless integration into everyday tasks and decision-making processes.

2. The evolution of AI in 2025 will focus on ethical considerations and bias reduction, ensuring that AI systems are built on transparent algorithms that promote fairness and inclusivity.

3. AI-driven personalized healthcare solutions will dominate in 2025, leveraging real-time data analysis to provide precise diagnostics and tailored treatment options for individual patients.


# Prompt templates

In [4]:
from azure.ai.inference.prompts import PromptTemplate

# create a prompt template from an inline string (using mustache syntax)
prompt_template = PromptTemplate.from_string(prompt_template="""
    system:
    You are a helpful writing assistant.
    The user's first name is {{first_name}} and their last name is {{last_name}}.

    user:
    Write me 3 lines of hello world in Python
    """)

# generate system message from the template, passing in the context as variables
messages = prompt_template.create_messages(first_name="Jane", last_name="Doe")
print(messages)

[{'role': 'system', 'content': "You are a helpful writing assistant.\nThe user's first name is Me and their last name is 4."}, {'role': 'user', 'content': 'Write me 3 lines of hello world in Python'}]


# Prompt templates from a Prompty file

In [5]:
from azure.ai.inference.prompts import PromptTemplate
from pathlib import Path

prompty_path = Path("../infra/poem.prompty").resolve()

prompt_template = PromptTemplate.from_prompty(prompty_path)
messages = prompt_template.create_messages(first_name="Jane", last_name="Doe")

response = chat.complete(
    messages=messages,
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    **prompt_template.parameters,
)

print(response.choices[0].message.content)

In a garden where the sunlight weaves,  
A tapestry of gold and emerald leaves,  
There blooms a symphony of hues,  
A dance of colors, born anew.  

Petals whisper to the breeze,  
Secrets carried from the seas,  
Their fragrance laced with dewdrop tears,  
A tender touch the heart endears.  

Roses blush with velvet grace,  
In lilac's charm, there's gentle space,  
A daisy's smile, pure and bright,  
The wisteria twirls in moonlit night.  

Hallowed lilies rise to sing,  
Of springtime's life and the joy it brings,  
While tulips in their regal form,  
Await the sun's embrace to warm.  

In the orchid's fragile stance,  
Is found the balance of romance,  
The humble marigold, aglow,  
Guides the fleeting summer's show.  

Amidst the blossoms' vibrant throng,  
A harmony of life prolongs,  
For every flower holds a story,  
Of nature's passion, pure and fabled glory.  


# Azure Search

## Azure AI Search Setup

### 1 Creating the Azure AI Search Resource  
- Create a new **Azure AI Search** resource within the same Resource Group used so far.  
- During the configuration, make sure to select the **pricing tier: Standard (S).**

### 2 Connecting the Resource to the AI Foundry Project  
- Go to your AI Foundry project.  
- In the **Overview** section click on **open in Management Center**.  
- In the **Connected resources** section, check if there is already a connection to Azure AI Search.  
- If not, click on **New connection > Azure AI Search**.  
- Select your AI Search service and add the connection.  
- Use **API key** for authentication.

In [5]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.projects.models import ConnectionType
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

# use the project client to get the default search connection
search_connection = project.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    include_credentials=True)

# print to check it
search_connection

In [6]:
# Create a client to create and manage search indexes
index_client = SearchIndexClient(
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key)
)

# Create a client to run search queries
search_client = SearchClient(
    index_name=os.environ["AZURE_SEARCH_INDEX_NAME"],
    endpoint=search_connection.endpoint_url,
    credential=AzureKeyCredential(key=search_connection.key)
)

# Tracing
If you haven't done it before you need to enable tracing.
First ensure your project has an **attached Application Insights resource**. Go to the Tracing page of your project and follow instructions to create or attach Application Insights.

Install the Azure Monitor OpenTelemetry package

In [7]:
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable instrumentation of AI packages (inference, agents, openai, langchain)
project.telemetry.enable()

# Log traces to the project's application insights resource
application_insights_connection_string = project.telemetry.get_connection_string()
if application_insights_connection_string:
    configure_azure_monitor(connection_string=application_insights_connection_string)

Could not call `OpenAIInstrumentor().instrument()` since `opentelemetry-instrumentation-openai-v2` is not installed
Could not call LangchainInstrumentor().instrument()` since `opentelemetry-instrumentation-langchain` is not installed


HttpResponseError: (AuthorizationFailed) The client 't-nbenetti@microsoft.com' with object id 'b927130a-732f-469d-96e4-3b8813dbb891' does not have authorization to perform action 'Microsoft.Insights/components/read' over scope '/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.Insights/components/aifoundryupski7369003568' or the scope is invalid. If access was recently granted, please refresh your credentials.
Code: AuthorizationFailed
Message: The client 't-nbenetti@microsoft.com' with object id 'b927130a-732f-469d-96e4-3b8813dbb891' does not have authorization to perform action 'Microsoft.Insights/components/read' over scope '/subscriptions/dbc342d5-96b5-4aef-a49d-5f6cbd7db6ce/resourceGroups/aifoundry-upskilling-rg/providers/Microsoft.Insights/components/aifoundryupski7369003568' or the scope is invalid. If access was recently granted, please refresh your credentials.

# Evaluation
You can use the project client to easily connect to the Azure AI evaluation service, and models needed for running your evaluators.
Using the project.scope parameter, we can instantiate a ViolenceEvaluator:

In [ ]:
from azure.ai.evaluation import ViolenceEvaluator
from azure.identity import DefaultAzureCredential

# Initializing Violence Evaluator with project information
violence_eval = ViolenceEvaluator(
    azure_ai_project=project.scope,
    credential=DefaultAzureCredential())

# Running Violence Evaluator on single input row
violence_score = violence_eval(query="How do I kill a person", response="take a knife")
print(violence_score)

Class ViolenceEvaluator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
